## 7. 二八轮动
对比创业板与沪深300指数的40天Mom，创业板强选Mom10因子，沪深强选价值因子。

In [1]:
# coding=utf-8
import numpy as np
import talib as ta
import pandas as pd
from datetime import datetime
from fxdayu_data import DataAPI

DataAPI.set_file('D:/PycharmProjects/Quant_Alpha/Data_Manager/Valuation_Selection/config.py')
gem_close = DataAPI.candle('sz399006','D','close', start=datetime(2010, 7, 1))
hs300_close = DataAPI.candle('sh000001', 'D', 'close', start=datetime(2010, 7, 1))
gem_mom = ta.ROCR100(gem_close.close.values, 20)
hs300_mom = ta.ROCR100(hs300_close.close.values, 20)
index_mom = pd.DataFrame({'gem_mom':gem_mom, 'hs300_mom':hs300_mom}, index=gem_close.index)
codes_hs300 = pd.read_excel('D:/PycharmProjects/Quant_Alpha/Data_Manager/Valuation_Selection/Value_Selection.xlsx')
codes_gem = pd.read_excel('D:/PycharmProjects/Quant_Alpha/Momentun_Hedge/GEM_MOM.xlsx')
codes_hs300.index = codes_hs300.pop('date')
codes_gem.index = codes_gem.pop('date')
print index_mom
# print codes_hs300
# print codes_gem

[Errno 2] No such file or directory: 'config.py'
                        gem_mom   hs300_mom
datetime                                   
2010-07-01 15:00:00         NaN         NaN
2010-07-02 15:00:00         NaN         NaN
2010-07-05 15:00:00         NaN         NaN
2010-07-06 15:00:00         NaN         NaN
2010-07-07 15:00:00         NaN         NaN
2010-07-08 15:00:00         NaN         NaN
2010-07-09 15:00:00         NaN         NaN
2010-07-12 15:00:00         NaN         NaN
2010-07-13 15:00:00         NaN         NaN
2010-07-14 15:00:00         NaN         NaN
2010-07-15 15:00:00         NaN         NaN
2010-07-16 15:00:00         NaN         NaN
2010-07-19 15:00:00         NaN         NaN
2010-07-20 15:00:00         NaN         NaN
2010-07-21 15:00:00         NaN         NaN
2010-07-22 15:00:00         NaN         NaN
2010-07-23 15:00:00         NaN         NaN
2010-07-26 15:00:00         NaN         NaN
2010-07-27 15:00:00         NaN         NaN
2010-07-28 15:00:00        

In [2]:
def init(context):
    context.hs300 = index_mom['hs300_mom'] > index_mom["gem_mom"]
    context.codes_hs300 = codes_hs300
    context.codes_gem = codes_gem

def calculate(context, date, name, per):
    codes = getattr(context, name).loc[date]
    codes = codes[codes.values].index
    return pd.Series(per/len(codes), codes)

def rebalance(context, holds):
    for stock in context.portfolio.positions:
        if stock not in holds.index:
            order_target_percent(stock, 0)

    for code, per in holds.iteritems():
        order_target_percent(code, per)


def handle_bar(context, bar_dict):
    if context.hs300[context.now]:
        holds = pd.concat(
                [calculate(context, context.now, "codes_hs300", 0.9*0.95),
                 calculate(context, context.now, "codes_gem", 0.1*0.95)],
        )
    else:
        holds = pd.concat(
                [calculate(context, context.now, "codes_gem", 0.9*0.95),
                 calculate(context, context.now, "codes_hs300", 0.1*0.95)]
        )

    rebalance(context, holds)

config = {
    "base": {
        "start_date": "2010-08-25",
        "end_date": "2017-05-01",
        "frequency": '1d',
        "securities": ['stock', 'future'],
        "stock_starting_cash": 10000000,
        "future_starting_cash": 10000000,
        "benchmark": "000300.XSHG",
        "strategy_file_path": os.path.abspath(__file__)
    },
    "extra": {
        "log_level": "verbose",
    },
    "mod": {
        "sys_analyser": {
            "report_save_path": '.',
            "enabled": True,
            "plot": True
        }
    }
}

rqalpha.run_func(init=init, handle_bar=handle_bar, config=config)